# Uruchomienie jointa w trybie prędkości

In [1]:
import can
import matplotlib as plt
import numpy as np
import os
import time
import math
from IPython.display import display, clear_output
from joint_can import *
%matplotlib inline


In [2]:
# Ustawienie adresu CAN na "nr_id"

def initialize_joint(bus, mode: int, nr_id):
    send_msg(0x1A0 + nr_id, None, bus)
    time.sleep(1)
    
    send_msg(0x1b0 + nr_id, [0,0,0,0,0,0], bus)
    time.sleep(0.1)
    send_msg(0x140 + nr_id, None, bus)
    time.sleep(0.1)
    send_msg(0x1a0 + nr_id, None, bus)
    time.sleep(1)
    
    if (send_msg(0x1F0 + nr_id, [mode, 2], bus).data[0] == 1):
        send_msg(0x110 + nr_id, [2], bus)
        time.sleep(1)
        send_msg(0x110 + nr_id, [3], bus).data[0]
        time.sleep(5)
        return True
    else:
        return False
      


In [3]:
# os.popen('/home/pi/mech_diagnostics/can_down.sh')
# time.sleep(2)
# os.popen('/home/pi/mech_diagnostics/can_up.sh')
# time.sleep(6)

In [4]:
bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=False)
nr_id = get_can_id(bus) #numer id jaki chcemy nadac do jointa
initialize_joint(bus,2,nr_id)

True

# Ustawienie prędkości

In [5]:
speed=2 #tutaj ustawiasz predkosc w rad/s

# Blok główny programu

In [6]:
#blok pracuje w pętli, aby zatrzymać jointa trzeba zatrzymać ten blok i uruchomić kolejny

# fig = plt.pyplot.figure()
# ax = fig.add_subplot(1, 1, 1) 

#torque zwracany co 1s w Nm
c_speed=0.
j_data=set_speed(c_speed,bus, nr_id)
data_t=get_temp(bus, nr_id)
spin = 1

while(spin):
    c_speed = speed
    j_data=set_speed(c_speed,bus, nr_id)
    data_t=get_temp(bus, nr_id)
    inc, abso = get_register_value_from_joint(bus,nr_id,195)
    diff = inc - abso
    print(f'\r tq: {j_data.torque:.3f}\tm_temp: {data_t.motor_temp:.3f}\tb_temp: {data_t.bearing_temp:.3f}\tenkoder_inc: {inc:.3f}\tenkoder_abs: {abso:.3f}\tabs_inc_diff: {diff:.3f}', end='')
    time.sleep(0.25)
set_speed(0,bus,0)

 tq: 26.522	m_temp: 24.000	b_temp: 22.000	enkoder_inc: 74.000	enkoder_abs: 74.000	abs_inc_diff: 0.00000000

KeyboardInterrupt: 

In [7]:
initialize_joint(bus,2,nr_id)

send_msg(0x1A0 + nr_id, None, bus)
time.sleep(1)
send_msg(0x1F0 + nr_id, [2,2], bus)
send_msg(0x110 + nr_id, [2], bus)
time.sleep(1)
send_msg(0x110 + nr_id, [3], bus)
time.sleep(2)


data=set_speed(0,bus,nr_id)
sgn=(int(data.position>0) - int(data.position<0))

data=set_speed(sgn*0.1,bus,nr_id)
print(data.position)
while(abs(data.position)>0.01):
    if (abs(data.position)>0.2):
        data=set_speed(sgn*-0.25,bus,nr_id)
    else:
        data=set_speed(sgn*-0.05,bus,nr_id)
    time.sleep(0.1)

send_msg(0x110 + nr_id, [2], bus)
print(data.position)

2.9902992252322043
0.006423740583285031
